In [ ]:
%%capture --no-display
%pip install requests
%pip install python-dotenv
%pip install numpy

In [31]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()

def read_input_url(url):
    headers = {
        'Cookie': f'session={os.getenv("SESSION_COOKIE")}'
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.text

def get_aoc_input(day):
    return read_input_url(f'https://adventofcode.com/2024/day/{day}/input')

In [32]:
"""Day 1 input"""
day_1_input = get_aoc_input(1)
 
left = []
right = []
for line in day_1_input.splitlines():
    left_num, right_num = line.split("   ")
    left.append(int(left_num))
    right.append(int(right_num))

In [33]:
"""Solution to Day 1 Puzzle 1"""
def solution_1_1(left, right):
    similarity_score = 0
    for i in range(len(left)):
        similarity_score += abs(left[i] - right[i])
    return similarity_score

solution_1_1(left, right)

29037157

In [34]:
"""Solution to Day 1 Puzzle 2"""
def solution_1_2(left, right):
    similarity_score = 0
    for i in range(len(left)):
        left_num = left[i]
        occurences = right.count(left_num)
        similarity_score += left_num * occurences if occurences > 0 else 0
    return similarity_score

solution_1_2(left, right)

19678534

In [35]:
"""Day 2 input"""
day_2_input = get_aoc_input(2)

# A report is a list of levels seperated by new line
# A level is a number, seperasted by space

list_of_reports = []
for line in day_2_input.splitlines():
    report = line.split(" ")
    report = [int(level) for level in report]
    list_of_reports.append(report)

In [36]:
"""Soltion to Day 2 Puzzle 1"""

def is_ascending(report):
    return all(report[i] <= report[i+1] for i in range(len(report)-1))
    
def is_descending(report):
    return all(report[i] >= report[i+1] for i in range(len(report)-1))
    
def diff_within(report, min, max):
    for i in range(len(report)-1):
        diff = abs(report[i] - report[i+1])
        if diff < min or diff > max:
            return False
            
    return True

def is_safe(report):
    if not (is_ascending(report) or is_descending(report)):
        return False
    return diff_within(report, 1, 3)

count_safe_reports = 0
for report in list_of_reports:
    if is_safe(report):
        count_safe_reports += 1

count_safe_reports

359

In [37]:
"""Soltion to Day 2 Puzzle 2"""

def is_safe_with_tolerance_of_one_level_removed(report):
    for i in range(len(report)):
        new_report = report[:i] + report[i+1:]
        if is_safe(new_report):
            return True
    return False

count_safe_reports_with_tolerance_of_one_removed = 0
for report in list_of_reports:
    if is_safe_with_tolerance_of_one_level_removed(report):
        count_safe_reports_with_tolerance_of_one_removed += 1

count_safe_reports_with_tolerance_of_one_removed

418

In [38]:
"""Day 3 input"""
day_3_input = get_aoc_input(3)

In [39]:
"""Solution to Day 3 Puzzle 1"""

import re

def parse(content, pattern):
    return re.findall(pattern, content)

def evaluated_exs(parsed_content):
    result = 0
    for match in parsed_content:
        result += int(match[0]) * int(match[1])
    return result

def extract_mul_instructions(content, pattern):
    parsed_content = parse(content, pattern)
    return evaluated_exs(parsed_content)


mul_pattern = r"mul\((\d+),(\d+)\)"
extract_mul_instructions(day_3_input, mul_pattern)

164730528

In [40]:
"""Solution to Day 3 Puzzle 2"""

def build_pattern():
    patterns = {
        'do': r'do\(\)',
        'dont': r'don\'t\(\)',
        'mul': r"mul\((\d+),(\d+)\)"
    }
    
    combined = '|'.join(f'(?P<{name}>{pattern})' 
                       for name, pattern in patterns.items())
    
    return re.compile(combined)


def extract_do_and_dont_mul_instructions(content):
    combined_pattern = build_pattern()
    pattern = re.compile(combined_pattern)
    
    enabled = True
    result = 0
    
    for match in pattern.finditer(content):
        if match.group('do'):
            enabled = True
        elif match.group('dont'):
            enabled = False
        elif match.group('mul') and enabled:
            a, b = map(int, match.group('mul')[4:-1].split(','))
            result += a * b
            
    return result
        

extract_do_and_dont_mul_instructions(day_3_input)

70478672

In [41]:
"""Day 4 input"""
day_4_input = get_aoc_input(4)

In [42]:
"""Solution to Day 4 Puzzle 1"""

import re
import numpy as np
from enum import Enum, auto


class Direction(Enum):
    HORIZONTAL = auto()
    VERTICAL = auto()
    LEFT_DIAGONAL = auto()
    RIGHT_DIAGONAL = auto()
    REVERSE = auto()


def create_matrix(input_str):
    return np.array([list(line) for line in input_str.splitlines()])

def get_all_lines(matrix, word_length, directions):
    rows, cols = matrix.shape
    lines = []

    if Direction.HORIZONTAL in directions:
        for row in matrix:
            lines.append("".join(row))

    if Direction.VERTICAL in directions:
        for col in matrix.T:
            lines.append("".join(col))

    if Direction.LEFT_DIAGONAL in directions:
        for i in range(-(rows - word_length), cols - word_length + 1):
            diag = np.diag(matrix, k=i)
            if len(diag) >= word_length:
                lines.append("".join(diag))

    if Direction.RIGHT_DIAGONAL in directions:
        flipped = np.fliplr(matrix)
        for i in range(-(rows - word_length), cols - word_length + 1):
            diag = np.diag(flipped, k=i)
            if len(diag) >= word_length:
                lines.append("".join(diag))

    if Direction.REVERSE in directions:
        lines.extend([line[::-1] for line in lines[:]])

    return lines


def count_word_occurrences(input_str, word, directions=None):
    matrix = create_matrix(input_str)
    lines = get_all_lines(matrix, len(word), directions)
    total = 0
    pattern = f"(?={word})"

    for line in lines:
        total += len(re.findall(pattern, line))

    return total

occurences = count_word_occurrences(
    day_4_input,
    "XMAS",
    {Direction.HORIZONTAL, Direction.REVERSE, Direction.LEFT_DIAGONAL, Direction.RIGHT_DIAGONAL, Direction.VERTICAL},
)

occurences

2718

In [43]:
"""Solution to Day 4 Puzzle 2"""

def get_crossed_diagonal_occurrences(content, word):
    matrix = np.array([list(line) for line in content.splitlines()])
    rows, cols = matrix.shape
    occurrences = 0

    word_len = len(word)
    center_idx = word_len // 2
    center_char = word[center_idx]
    pattern = f"(?={word})"

    # Find all center character positions
    center_positions = np.argwhere(matrix == center_char)

    for pos in center_positions:
        i, j = pos

        # Skip if too close to edges
        if (
            i < center_idx
            or i >= rows - center_idx
            or j < center_idx
            or j >= cols - center_idx
        ):
            continue

        # Extract longer diagonals for pattern matching
        left_diag = "".join(
            matrix[
                i - center_idx : i + center_idx + 1, j - center_idx : j + center_idx + 1
            ].diagonal()
        )
        right_diag = "".join(
            np.fliplr(
                matrix[
                    i - center_idx : i + center_idx + 1,
                    j - center_idx : j + center_idx + 1,
                ]
            ).diagonal()
        )

        # Check both directions in each diagonal
        left_matches = len(re.findall(pattern, left_diag)) + len(
            re.findall(pattern, left_diag[::-1])
        )
        right_matches = len(re.findall(pattern, right_diag)) + len(
            re.findall(pattern, right_diag[::-1])
        )

        if left_matches > 0 and right_matches > 0:
            occurrences += 1

    return occurrences


get_crossed_diagonal_occurrences(day_4_input, "MAS")

2046

In [44]:
"""Day 5 input"""
day_5_input = get_aoc_input(5)

In [45]:
"""Solution to Day 5 Puzzle 1"""

rule_section, number_section = day_5_input.split("\n\n")
rules = [list(map(int, section.split("|"))) for section in rule_section.splitlines()]
numbers = [list(map(int, section.split(","))) for section in number_section.splitlines()]

def is_ordered(rules: list[int], numbers: list[int]):
    for rule in rules:
        left, right = rule
        left_index = numbers.index(left) if left in numbers else -1
        right_index = numbers.index(right) if right in numbers else -1
        if left_index != -1 and right_index != -1 and left_index > right_index:
            return False
    return True

def get_correctly_ordered_numbers(rules: list[int], numbers_list: list[list[int]]):
    for numbers in numbers_list:
        if is_ordered(rules, numbers):
            yield numbers


def get_center_number(numbers_list: list[list[int]]):
    for numbers in numbers_list:
        center = len(numbers) // 2
        yield (numbers[center])

correct_ordered_numbers = list(get_correctly_ordered_numbers(rules, numbers))

sum(get_center_number(correct_ordered_numbers))

6242

In [46]:
"""Solution to Day 5 Puzzle 2"""

def order_numbers(rules: list[int], numbers: list[int]):
    copylist = numbers.copy()
    get_relevant_rules = [rule for rule in rules if rule[0] in copylist and rule[1] in copylist]
    for _ in copylist:
        for rule in get_relevant_rules:
            left, right = rule
            left_index = copylist.index(left)
            right_index = copylist.index(right)

            if left_index > right_index:
                copylist[left_index], copylist[right_index] = copylist[right_index], copylist[left_index]

    return copylist

def order_all_unordered_numbers(rules: list[int], numbers_list: list[list[int]]):
    for numbers in numbers_list:
        if not is_ordered(rules, numbers):
            yield order_numbers(rules, numbers)


sum(get_center_number(order_all_unordered_numbers(rules, numbers)))

5169

In [47]:
"""Day 6 input"""
day_6_input = get_aoc_input(6)

In [48]:
"""Solution to Day 6 Puzzle 1"""

directions = [">", "v", "<", "^"]
obstacle = "#"
empty = "."


def get_guard_pos_and_dir(matrix):
    for i, row in enumerate(matrix):
        for j, cell in enumerate(row):
            if cell in directions:
                return i, j, cell


def peek(matrix, i, j, direction):
    row, col = matrix.shape
    next_i, next_j = i, j

    if direction == ">":
        next_j += 1
    elif direction == "v":
        next_i += 1
    elif direction == "<":
        next_j -= 1
    elif direction == "^":
        next_i -= 1

    if next_i < 0 or next_i >= row or next_j < 0 or next_j >= col:
        return None
    return matrix[next_i, next_j]


def new_direction(current_direction):
    if current_direction == "^":
        return ">"
    if current_direction == ">":
        return "v"
    if current_direction == "v":
        return "<"
    if current_direction == "<":
        return "^"


def move(matrix, guard):
    i, j, direction = guard
    peek_ahead = peek(matrix, i, j, direction)
    if peek_ahead == None:
        return i, j, None
    if peek_ahead == obstacle:
        return i, j, new_direction(direction)
    if direction == "^":
        return i - 1, j, direction
    if direction == ">":
        return i, j + 1, direction
    if direction == "v":
        return i + 1, j, direction
    if direction == "<":
        return i, j - 1, direction


def guard_route(matrix, guard):
    i, j, direction = guard
    route_matrix = np.zeros_like(matrix, dtype=int)
    route_matrix[i, j] = 1

    while True:
        cur_i, cur_j = i, j
        guard = move(matrix, guard)
        i, j, direction = guard

        route_matrix[cur_i, cur_j] += 1

        if direction == None:
            return route_matrix


matrix = create_matrix(day_6_input)
guard = get_guard_pos_and_dir(matrix)
route = guard_route(matrix, guard)
sum(map(lambda x: 1 if x > 0 else 0, route.flatten()))

5242

In [49]:
"""Solution to Day 6 Puzzle 2"""

def is_loop(matrix, guard):
    visited = set()
    i, j, direction = guard

    while True:
        state = (i, j, direction)
        if state in visited:
            return True
        
        visited.add(state)
        guard = move(matrix, guard)
        i, j, direction = guard

        if direction == None:
            return False

def find_all_loops(matrix, guard):
    empty_positions = np.argwhere(matrix == empty)
    loops = []
    matrix_copy = matrix.copy()

    for i, j in empty_positions:
        matrix_copy[i, j] = obstacle
        if is_loop(matrix_copy, guard):
            loops.append((i, j))
        matrix_copy[i, j] = empty

    return loops

matrix = create_matrix(day_6_input)
guard = get_guard_pos_and_dir(matrix)
len(list(find_all_loops(matrix, guard)))

1424

In [50]:
"""Day 7 input"""
day_7_input = get_aoc_input(7)

In [51]:
"""Solution to Day 7 Puzzle 1"""

from itertools import product

operands = {
    "+": lambda a, b: a + b,
    "*": lambda a, b: a * b,
}

def split_expression(content):
    expect, remainder = content.split(":")
    expect = int(expect)
    numbers = list(map(int, remainder.strip().split(" ")))
    return expect, numbers


def get_expression(input):
    return map(split_expression, input.splitlines())


def evaluate_expression(numbers, perm, expect):
    result = numbers[0]
    for i, number in enumerate(numbers[1:]):
        if operands.get(perm[i]):
            result = operands[perm[i]](result, number)
        else:
            raise ValueError(f"Invalid operand {perm[i]}")
        
        if result > expect:
            return None

    return result


def get_correct_expression(expect: int, numbers: list[int]):
    for perm in product(operands.keys(), repeat=len(numbers) - 1):
        if evaluate_expression(numbers, perm, expect) == expect:
            return expect


def get_valid_values(expressions: list[int, list[int]]):
    for expect, numbers in expressions:
        result = get_correct_expression(expect, numbers)
        if result:
            yield result


expressions = get_expression(day_7_input)
correct_expressions = list(get_valid_values(expressions))
sum(correct_expressions)

5837374519342

In [52]:
"""Solution to Day 7 Puzzle 2"""

operands["||"] = lambda a, b: int(f"{a}{b}")

expressions = get_expression(day_7_input)
correct_expressions = list(get_valid_values(expressions))
sum(correct_expressions)

492383931650959

In [53]:
"""Day 8 input"""
day_8_input = get_aoc_input(8)

In [133]:
"""Day 8 Puzzle 1"""


def parse_input(content):
    lines = content.splitlines()
    width = len(lines[0])
    height = len(lines)
    return lines, width, height


def make_points(content, width, height):
    points = []
    for y in range(height):
        for x in range(width):
            if content[y][x] != ".":
                points.append((content[y][x], x, y))
    return points


def within(width, height, x, y) -> bool:
    return 0 <= x < width and 0 <= y < height


def diff(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return dx, dy


def diff_offset(x1, y1, x2, y2):
    dx, dy = diff(x1, y1, x2, y2)
    return x2 + dx, y2 + dy


def antinodes(points: list):
    new_points = set()
    for a in points:
        for b in points:
            if a[0] != b[0]:
                new_points.add(b)
                continue

            x, y = diff_offset(a[1], a[2], b[1], b[2])

            if (a[0], x, y) in points:
                continue

            if within(width, height, x, y):
                new_points.add(("#", x, y))

    return new_points


lines, width, height = parse_input(day_8_input)
points = make_points(lines, width, height)
anti = antinodes(points)
count = len(list(filter(lambda x: x[0] == "#", anti)))
count

390

In [134]:
"""Day 8 Puzzle 2"""


def blocked(points, x, y) -> bool:
    return any(px == x and py == y for _, px, py in points)


def antinodes(points: list, width: int, height: int):
    anti = set()
    for a in points:
        for b in points:
            if a == b or a[0] != b[0]:
                continue

            dx, dy = diff(a[1], a[2], b[1], b[2])
            x, y = diff_offset(a[1], a[2], b[1], b[2])

            while within(width, height, x, y):
                if not blocked(points, x, y):
                    anti.add(("#", x, y))
                x += dx
                y += dy

    return anti


lines, width, height = parse_input(day_8_input)
points = make_points(lines, width, height)
anti = antinodes(points, width, height)
count = len(anti) + len(points)
count

1246